# Experimenting With 1D CNN on Baumhofer Data
All models and their variants should be stored in ./ml_utils/models.py<br>
Opening a new notebook for working with the model, separate to the data pre-processing, helps keep things clean and easily accessible for the future.

In [18]:
import numpy as np
import pickle
import pandas as pd

from tensorflow.keras.callbacks import LearningRateScheduler

from ml_utils import models, tools

In [13]:
# Make some fake data
X_train_sc = np.random.random((4000, 700, 3))
y_train = np.random.random((4000, 1))

In [12]:
model = models.build_convnet_model(X_in=X_train_sc, loss='mse', n_outputs=1)
#model.summary()

In [19]:
verbose = 1
callbacks = [LearningRateScheduler(tools.lr_scheduler, verbose=0)]

history = model.fit(X_train_sc, 
                    y_train,
                    callbacks=callbacks,
                    batch_size=512,
                    epochs=200,    
                    shuffle=True,
                    verbose=verbose)

### Load pre-processed Baumhofer data from file

In [17]:
# ...

### Train the model

In [ ]:
model = build_convnet_model(n_outputs=1)

target_idx = 2
verbose = 0
callbacks = [LearningRateScheduler(lr_scheduler, verbose=verbose)]
history = model.fit(X_train_sc, 
                    y_train[:, target_idx],
                    callbacks=callbacks,
                    batch_size=512,
                    epochs=200,    
                    shuffle=True,
                    verbose=verbose)

### Evaluate the trained model on train, val and test

In [ ]:
model.evaluate(X_train_sc, y_train[:, target_idx])

In [ ]:
model.evaluate(X_test_sc, y_test[:, target_idx])

In [ ]:
model.evaluate(X_val_sc, y_val[:, target_idx])

### Train predictions

In [ ]:
plt.figure(figsize=(20,5))

plt.plot(model.predict(X_train_sc))
plt.plot(y_train[:, target_idx])

### Val predictions

In [ ]:
plt.figure(figsize=(20,5))

plt.plot(model.predict(X_val_sc))
plt.plot(y_val[:, target_idx])

### Test predictions

In [ ]:
plt.figure(figsize=(20,5))

plt.plot(model.predict(X_test_sc))
plt.plot(y_test[:, target_idx])